 Reusable Software for Assortment Planning

Create a function called "optimizeAssortment" with two input arguments:

- inputFile: filename of the input file. See the attached "PS10-books-input-1.xlsx" and "PS10-books-input-2.xlsx" for examples of the sample input.
- outputFile: filename of the output file that the function will create. 

The function should implement the abstract formulation for Problem 9.2 and produce the outputFile. See the attached "PS10-books-sampleOutput-1.xlsx" for the desired output file format corresponding to the "PS10-books-input-1.xlsx" input file. The abstract formulation is reproduced below for your convenience.

**Data:**

- $B$: the set of books.
- $G$: the set of genres.
- $a_{bg}$: whether book $b$ is in genre $g$.
- $q_g$: how many books we need of genre $g$.

**Decision Variables:** Let $x_b$ deibite whether to carry book $b$. (Binary)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize:} && \sum_{b \in B} x_b \\
\text{subject to:} \\
\text{(Enough books in genre)} && \sum_{b \in B} a_{bg}x_b & \ge q_g & \text{ for each genre $g \in G$.}
\end{aligned}$$


In [401]:
# Write your code here
from gurobipy import Model, GRB
import pandas as pd
def optimizeAssortment(inputFile, outputFile):
    genres = pd.read_excel(inputFile, index_col = 0)
    genres = genres.fillna(0)
    requirements = pd.read_excel(inputFile, sheet_name='requirements', index_col=0)
    B = genres.index
    G = requirements.index
    a = genres
    q = requirements
    
    #Model
    mod = Model()
    x = mod.addVars(B, name = 'x', vtype = GRB.BINARY)
    mod.update()
    mod.setObjective(sum(x[b] for b in B), GRB.MINIMIZE)
    for g in G:
        mod.addConstr(sum(a.loc[b,g]*x[b] for b in B) >= q.loc[g,'required'])
    mod.setParam('OutputFlag',False)
    mod.optimize()
    print('Finished optimizing! Objective value:',mod.objval)
    
    #Excel
    writer = pd.ExcelWriter(outputFile)
    
    # Writing summary sheet
    bo = []
    for i in B:
        if x[i].X >0:
            bo.append(i)

    di = {'books': bo}
    df = pd.DataFrame(di)
    df.to_excel(writer,sheet_name='optimal_decision')

    # Writing solution sheet
    pd.DataFrame([mod.objval],columns = ['books_needed']).to_excel(writer,sheet_name='objective',index=False)
    writer.save() 

In [402]:
# Test code (will create two output files for the two input files)
optimizeAssortment('PS10-books-input-1.xlsx','PS10-books-output-1.xlsx') 
optimizeAssortment('PS10-books-input-2.xlsx','PS10-books-output-2.xlsx') 

Finished optimizing! Objective value: 4.0
Finished optimizing! Objective value: 154.0


Assigning of Final Grades

This question asks you to create software that can help a professor assign final grades in such a way so that the average GPA rounds to 3.5, while obtaining an assignment in which there are gaps in scores between consecutive grade levels, and no particular grade is assigned to disproportionally many students.

**Data:** 

- $I$: the set of students.
- $n$: the number of students.
- $J=\{0,1,\cdots\}$: numerical indices used to denote the various grade levels.
- $s_i$: the overall score of student $i\in I$ (between 0 and 100). 
- $g_j$: the GPA corresponding to grade level $j \in J$.

**Decision Variables:**

- $x_{ij}$: whether to assign student $i$ to grade level $j$. (Binary)
- $t_j$: the number of students assigned to grade level $j$. (Continuous)
- $L_j$: the score cutoff for grade level $j$. (Continuous)
- $U_j$: the maximum score in grade level $j$. (Continuous)

$$\begin{aligned}
\text{Min} && \sum_{j \in J}(U_j-L_j) + 0.1 \sum_{j \in J} t_j \times t_j \\
\text{s.t.} \\
\text{(Average GPA)} && 3.495n \le \sum_{i \in I}\sum_{j \in J} x_{ij}g_{j} & \le 3.505n \\
\text{(Assignment)} && \sum_{j \in J} x_{ij} & = 1 && \text{for each $i \in I$.}\\
\text{(Max score)} && s_i x_{ij} & \le U_j && \text{for each $i \in I$, $j \in J$.}\\
\text{(Min score)} && 100(1-x_{ij}) + s_i x_{ij} & \ge L_j && \text{for each $i \in I$, $j \in J$.}\\
\text{(Correct totals)} && \sum_{i \in I} x_{ij} & = t_j && \text{for each $j \in J$.}\\
\text{(Bounds)} && L_j & \le U_j && \text{for each $j \in J$.} \\
\text{(Ordering)} && U_j & \le L_{j-1} && \text{for each $j \in J$ with $j \ge 1$.}
\end{aligned}$$

The input data is contained in an Excel file named `PS10-grade-input.xlsx` with two sheets. The first sheet, named "Scores", contains the score of each student. The first five entries look like:

![Sample Scores Sheet](PS10-grade1.png)

The second sheet, named "Levels", is as follows

![Sample Levels Sheet](PS10-grade2.png)

The output data should be an Excel file named `PS10-grade-output.xlsx` that contains the cutoff ($L_j$) for each grade level $j$. It should look like

![Sample Output](PS10-grade3.png)



In [403]:
# Write your code here

from gurobipy import Model, GRB
import pandas as pd
def gpacalculator(inputFile, outputFile):
    
    g = pd.read_excel(inputFile, index_col=0)
    l = pd.read_excel(inputFile, sheet_name='Levels',index_col=0)
    I = g.index
    n = len(I)
    J = l.index
    s = g
    g = l['g_j']

    #Model
    mod = Model()
    x = mod.addVars(I,J, name = 'x', vtype = GRB.BINARY)
    t = mod.addVars(J, name = 't')
    L = mod.addVars(J, name = 'L')
    U = mod.addVars(J, name = 'U')
    mod.setObjective(sum(U[j] - L[j] for j in J) + 0.1*sum(t[j]*t[j] for j in J))
    mod.addConstr(sum(x[i,j]*g[j] for i in I for j in J) <= 3.505*n)
    mod.addConstr(sum(x[i,j]*g[j] for i in I for j in J) >= 3.495*n)
    for i in I:
        mod.addConstr(sum(x[i,j] for j in J) == 1)
    for i in I:
        for j in J:
            mod.addConstr(s.loc[i,'s_i']*x[i,j] <= U[j])
    for i in I:
        for j in J:
            mod.addConstr(100*(1-x[i,j]) + s.loc[i,'s_i']*x[i,j] >= L[j])
    for j in J:
        mod.addConstr(sum(x[i,j] for i in I) == t[j])
    for j in J:
        mod.addConstr(L[j] <= U[j])
    for j in J:
        if j >=1:
            mod.addConstr(U[j] <= L[j-1])
    mod.update()
    mod.setParam('OutputFlag',False)
    mod.optimize()

    #Excel

    # Opening excel file
    writer = pd.ExcelWriter(outputFile)

    # Writing solution sheet
    gr = []
    for j in J:
        gr.append([l.loc[j,'Letter'], int(round(L[j].x,0))])   
    df = pd.DataFrame(gr, columns = ['Letter', 'Cutoff'])
    df.to_excel(writer,index=False)
    writer.save()


In [404]:
gpacalculator('PS10-grade-input.xlsx', 'PS10-grade-output.xlsx')

 Team Assignment

The following MIP can used to assign students into project teams to balance the overall characteristics of each team.

**Data:**

- $I$: set of students.
- $n$: number of teams
- $J=\{1,2,\cdots,n\}$ : set of teams.
- $K$: set of characteristics.
- $a_{ik}$: student $i$'s value for characteristics $k$.
- $w_k$: the weight for characteristics $k$ in the objective.
- $L_k$: the ideal lower bound for the sum of characteristic $k$ for any team. 
- $U_k$: the ideal upper bound for the sum of characteristics $k$ for any team.

You should assume that the data is given in a excel file with the same format as the `PS10-Team-input-1.xlsx` and `PS10-Team-input-2.xlsx` files attached to this assignment. 

The sheet named "Students" encodes $I$, $K$ and $a_{ik}$'s. In the below screenshot of `PS10-Team-input-1.xlsx`, $I=\{A,B,C,D,E,F\}$, and $K=\{Person, Male, Programmer, Math, Speaking\}$.

![](PS10-Team1.png)


The sheet named "Parameters" encodes the $w_k$, $L_k$ and $U_k$ for each characteristic $k$.

![](PS10-Team2.png)

**Decision variables:**

- $x_{ij}$ : whether to assign student $i$ to team $j$. (Binary)
- $s_k$ : maximum deviation below the ideal lower bound $L_k$ for characteristic $k$. (Continuous)
- $t_k$ : maximum deviation above the ideal upper bound $U_k$ for characteristic $k$. (Continuous)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize:} && \sum_{k \in K} w_k(s_k+t_k) \\
\text{subject to:} && \\
\text{(Every person assigned)} && \sum_{j \in J} x_{ij} & = 1 && \text{For each person $i \in I$.}\\
\text{(Team balance)} && L_k - s_k \le \sum_{i \in I} a_{ik}x_{ij} & \le U_k + t_k && \text{For each team $j \in J$ and each $k \in K$.} \\
\text{(Non-negativity)} && s_k, t_k & \ge 0 && \text{for all $k$.}
\end{aligned}$$

**Write a function called "assignTeams" with the following input arguments:**

- **inputFile:** path to the input spreadsheet.
- **n:** the number of teams to divide students into.

**The function should return two variables:**

- **df:** a DataFrame with one column called "Team". The index should be the name of each individual, and the column "Team" should specify the number $j$ to which the person is assigned.
- **objval:** the optimal objective value.

For the test runs, you should download the input files attached to this exercise into the same directory as the Jupyter notebook.

In [405]:
# Write your final code here
from gurobipy import Model, GRB
import pandas as pd
def assignTeams(inputFile, n):
    s = pd.read_excel(inputFile, index_col=0)
    p = pd.read_excel(inputFile, sheet_name= 'Parameters', index_col=0)
    I = s.index
    K = s.columns
    a = s
    w = p.loc['Weights']
    L = p.loc['L']
    U = p.loc['U']
    w['Person']
    n = range(1,n+1)
    J = []
    for e in n:
        J.append(e)
    
    #Model
    
    mod = Model()
    x = mod.addVars(I,J, name = 'x', vtype = GRB.BINARY)
    s = mod.addVars(K, name = 's')
    t = mod.addVars(K, name = 't')
    mod.setObjective(sum(w[k]*(s[k]+t[k]) for k in K), GRB.MINIMIZE)
    for i in I:
        mod.addConstr(sum(x[i,j] for j in J) ==1)
    for j in J:
        for k in K:
            mod.addConstr(sum(a.loc[i,k]*x[i,j] for i in I) <= U[k] + t[k])    
    for j in J:
        for k in K:
            mod.addConstr(sum(a.loc[i,k]*x[i,j] for i in I) >= L[k] - s[k])
    mod.update()
    mod.setParam('OutputFlag',False)
    mod.optimize()
    objval = mod.objVal
    l = []
    for i in I:
        for j in J:
            if x[i,j].X >0:
                l.append([i, j])
    df = pd.DataFrame(l, columns=['Names', 'Team'])
    df.set_index('Names', inplace=True)
    return df, objval

In [406]:
df,objval=assignTeams('PS10-Team-input-1.xlsx',2)
print('Optimal objective value:',objval)
df

Optimal objective value: 0.0


,Team
Names,
A,2
B,2
C,2
D,1
E,1
F,1


In [407]:
df,objval=assignTeams('PS10-Team-input-2.xlsx',10)
print('Optimal objective value:',objval)
df.sort_values(by='Team')

Optimal objective value: 8.0


,Team
Names,
Bob,1
Yingying,1
Xenna,1
Wing,1
I-Ting,1
Patty,1
Ran,2
Qinan,2
Kathryn,2
